In [1]:
import torch
import torchvision.transforms as T
from dataset import *

# Sample a dataset and save it

In [2]:
# Set distributions of factors' values. If not specified, we will sample factors randomly
dist={'shape': [1, 2, 1, 4],
      'object_hue': [1, 1, 1, 2, 2, 3, 1, 3, 3, 1]
}

In [3]:
# sample a dataset and save
sample_3dshape_dataset(dist, data_size=5000, save_name='example_set')

# Use the sampled dataset

In [4]:
transform = T.ToTensor()

In [5]:
# _FACTORS_IN_ORDER = ['floor_hue', 'wall_hue', 'object_hue', 'scale', 'shape', 'orientation']
# concept_interest is the indices of the interested factors that will be labels
train_dataset = ShapesDataset(data_path='example_set.npy', concept_interest=[2,4], transform=transform)

In [6]:
len(train_dataset)

5000

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=1)

In [8]:
len(train_loader)

79

In [9]:
train_iter = iter(train_loader)

In [10]:
images, labels = train_iter.next()

In [11]:
images.shape

torch.Size([64, 3, 64, 64])

In [12]:
labels.shape

torch.Size([64, 2])

In [13]:
labels[0]

tensor([0.8000, 0.0000])

Or we use data_prefetcher

In [14]:
class data_prefetcher():
    def __init__(self, loader):
        self.loader = iter(loader)
        self.stream = torch.cuda.Stream()
        self.preload()

    def preload(self):
        try:
            self.next_input, self.next_target = next(self.loader)
        except StopIteration:
            self.next_input = None
            self.next_target = None
            return

        with torch.cuda.stream(self.stream):
            self.next_input = self.next_input.cuda(non_blocking=True)
            self.next_target = self.next_target.cuda(non_blocking=True)

    def next(self):
        torch.cuda.current_stream().wait_stream(self.stream)
        input = self.next_input
        target = self.next_target
        if input is not None:
            input.record_stream(torch.cuda.current_stream())
        if target is not None:
            target.record_stream(torch.cuda.current_stream())
        self.preload()
        return input, target

In [15]:
prefetcher = data_prefetcher(train_loader)

In [16]:
inputs, target = prefetcher.next()

In [17]:
inputs.shape

torch.Size([64, 3, 64, 64])

In [18]:
target.shape

torch.Size([64, 2])

In [19]:
target[20]

tensor([0.6000, 3.0000], device='cuda:0')

In [20]:
inputs[20]

tensor([[[0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
         [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
         [0.6000, 0.6000, 0.6000,  ..., 0.6000, 0.6000, 0.6000],
         ...,
         [0.2314, 0.2314, 0.2314,  ..., 0.1804, 0.1765, 0.1765],
         [0.2275, 0.2275, 0.2275,  ..., 0.1804, 0.1804, 0.1765],
         [0.2235, 0.2235, 0.2235,  ..., 0.1804, 0.1804, 0.1765]],

        [[0.8863, 0.8863, 0.8863,  ..., 0.8863, 0.8863, 0.8863],
         [0.8863, 0.8863, 0.8863,  ..., 0.8863, 0.8863, 0.8863],
         [0.8863, 0.8863, 0.8863,  ..., 0.8863, 0.8863, 0.8863],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.9765, 0.9765, 0.9765,  ..., 0.9765, 0.9765, 0.9765],
         [0.9765, 0.9765, 0.9765,  ..., 0.9765, 0.9765, 0.9765],
         [0.9765, 0.9765, 0.9765,  ..., 0.9765, 0.9765, 0.